In [ ]:
import pupil_classv2 as pc
import math
from statistics import mean
import csv
from timestamps import new_timestamps
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import *
import tdt
from scipy.signal import find_peaks, peak_widths,resample,correlate, correlation_lags,peak_prominences,resample,butter, lfilter, freqz,medfilt
from scipy.stats import pearsonr,wilcoxon,mannwhitneyu,kruskal
import os
from fp_helper import *
from matplotlib import rcParams
import cv2
from scipy.stats import ttest_ind,ttest_rel,zscore
from sklearn.linear_model import Ridge,RidgeCV
from sklearn.model_selection import KFold,cross_val_score
from random import randint
from scipy.ndimage import median_filter
import scikit_posthocs as sp
import statsmodels.api as sm 
from statsmodels.formula.api import ols 

params = {
        "font.family" : "Arial",
        'pdf.fonttype' : 42,
        'axes.labelsize': 10,
        'axes.titlesize': 11,
        'axes.linewidth': 0.5,
        'xtick.labelsize':9,
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'ytick.labelsize':9,
        'axes.spines.top':False,
        'axes.spines.right':False
         }
rcParams['figure.figsize'] = 21.7,8.27
rcParams.update(params)
def format_ax(ax,xlim,ylim,xspace,yspace):
   

    ax.set_xticks(np.arange(xlim[0],xlim[1]+ xspace,xspace))
    ax.set_yticks(np.arange(ylim[0],ylim[1]+ yspace,yspace))

    ax.set(ylim=(ylim[0], ylim[1]))
    ax.set(xlim=(xlim[0], xlim[1]))


In [ ]:
###low pass filter pupil
def filter_pupil(pupil,order = 1,cutoff = 1,fs = 20):
    def butter_lowpass(cutoff, fs, order=5):
        return butter(order, cutoff, fs=fs, btype='low', analog=False)

    def butter_lowpass_filter(data, cutoff, fs, order=5):
        b, a = butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y

    return butter_lowpass_filter(pupil, cutoff, fs, order)

In [ ]:
### get pupil events
###get events from all sessions

def get_pupil_events(id,date):
    id = id
    date = date
    face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
    path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
    df = pd.read_csv(path)
    pupil = np.array(df["Pupil Size"])
    time = np.array(df["Time"])
    speed = np.array(df["Running Speed"])
    speed[0] = 0
    dff = np.array(df["dFF"])
    face = np.array(pd.read_csv(face_path)["Facial Movement"])

    ###Onset detection alogrithm

    
    filter = filter_pupil(pupil) ###first apply low pass filter to pupil trace

    def groupSequence(x): 
        it = iter(x) 
        prev, res = next(it), [] 
    
        while prev is not None: 
            start = next(it, None) 
    
            if start and start > prev:

                res.append(prev) 
            elif res: 
                yield list(res + [prev]) 
                res = [] 
            prev = start 


    regions = list(groupSequence(filter))
    corrected_regions = [region for region in regions if len(region) >=15 ] ###only include regions that are >750ms
    corrected_regions = [region for region in corrected_regions if region[0]<np.median(filter) ] ###only include onsets that occur below z = median

    region_ix = [[np.where(filter == val)[0][0] for val in region] for region in corrected_regions] ### get indices for each region


    ##join regions that are near eachother
    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>20:
                if start-end<= 20:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]


    new = []
    for i,first in enumerate(region_ix):
        if i < len(region_ix)-1:
            second = region_ix[i + 1]
            end = first[-1]
            start = second[0]
            if first[0] - region_ix[i - 1][-1]>10:
                if start-end<= 10:
                    #print(first[0]/20,second[0]/20)
                    new.append(first + second)
                else:
                    new.append(first)
    region_ix = new
    corrected_regions = [filter[region[0]:region[-1]] for region in region_ix]
    onsets_ix = [region[0] for region in region_ix]


    ###get offsets and durations for pupil event
    offset_ix = []
    durations = []
    for i,(region,index) in enumerate(zip(corrected_regions,region_ix)):
        if i < len(region_ix)-1:
            start = index[-1] ###end of region
            end = region_ix[i+1][0] ### start of next region
            onset_y = region[0]
            offsets = np.where(filter[start:end] <= onset_y) ###find ix of first point that goes below onset 
        if len(offsets[0]) == 0: ###if it never goes below onset then use lowest value
            offset = start + np.argmin(filter[start:end])
        else:
            offset = start + offsets[0][0]


        offset_ix.append(offset)

        durations.append((offset-index[0])/20)

    ###handle last pupil event
    start = region_ix[-1][-1]
    end = 36000
    onset_y = corrected_regions[-1][0]
    offsets = np.where(filter[start:end] <= onset_y)
    if len(offsets[0]) == 0:
        offset = end-1
    else:
        offset = start + np.argmin(filter[start:end])

    offset_ix[-1] = offset

    durations[-1] = (offset-region_ix[-1][0])/20

    ###get pupil event amplitudes
    amplitudes = []
    amplitudes_ix = []
    for on,off in zip(onsets_ix,offset_ix):
        #print(on,off)
        amplitudes.append(max(filter[on:off])-filter[on])
        amplitudes_ix.append(on + np.argmax(filter[on:off]))

    ###get pupil event slopes
    slopes = []
    for region in corrected_regions:
        run = len(region)/20
        rise = region[-1]-region[0]
        slopes.append(rise/run)

    ###get dff data
    aucs = []
    peak_dffs = []
    mean_dffs = []
    for onset,offset in zip(onsets_ix,offset_ix):
        peak_dffs.append(max(dff[onset:offset]) - np.mean(dff[onset-30:onset]))
        aucs.append(np.trapz(dff[onset:offset]))
        mean_dffs.append(np.mean(dff[onset:offset])- np.mean(dff[onset-20:onset]))

    ###create dictionary

    events_dict = {"durations" :durations,
    "amplitudes": amplitudes,
    "slopes" : slopes,
    "onsets" : onsets_ix,
    "offsets" : offset_ix,
    "peak_ix" : amplitudes_ix,
    "ID" : [id] * len(durations),
    "Date" : [date] * len(durations),
    "peak_dff" : peak_dffs,
    "aucs" : aucs,
    "mean_dff" : mean_dffs,


    }
    

    return pd.DataFrame.from_dict(events_dict)


In [ ]:
###get quiet regions
def contiguous_regions(condition):
    """Finds contiguous True regions of the boolean array "condition". Returns
    a 2D array where the first column is the start index of the region and the
    second column is the end index."""

    # Find the indicies of changes in "condition"
    d = np.diff(condition)
    idx, = d.nonzero() 

    # We need to start things after the change in "condition". Therefore, 
    # we'll shift the index by 1 to the right.
    idx += 1

    if condition[0]:
        # If the start of condition is True prepend a 0
        idx = np.r_[0, idx]

    if condition[-1]:
        # If the end of condition is True, append the length of the array
        idx = np.r_[idx, condition.size] # Edit

    # Reshape the result into two columns
    idx.shape = (-1,2)
    return idx




def get_movement_events(events,speed):
    onsets = [on for on in events["onsets"]] ###get pupil onsets ix
    offsets = [off for off in events["offsets"]] ###get pupil offsets ix
    events_ix = []
    for i,(on,off) in enumerate(zip(onsets,offsets)):
        if np.mean(speed[on:off]) >np.percentile(speed,65):
            events_ix.append(i)

    return events_ix
    
def get_quiet_events(events,speed):
    speed = abs(speed)
    onsets = [on for on in events["onsets"]] ###get pupil onsets ix
    offsets = [off for off in events["offsets"]] ###get pupil offsets ix
    events_ix = []
    for i,(on,off) in enumerate(zip(onsets,offsets)):
        if np.mean(speed[on:off]) <np.percentile(speed,45):
            events_ix.append(i)
     
    return events_ix


In [ ]:
###get example trace data
id = "004116"
date = "20230815"
face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
df = pd.read_csv(path)
pupil = np.array(df["Pupil Size"])
time = np.array(df["Time"])
speed = zscore(np.array(df["Running Speed"]))
dff = np.array(df["dFF"])
face = np.array(pd.read_csv(face_path)["Facial Movement"])
df["Facial Movement"] = face
df["Running Speed"] = speed

events = get_pupil_events(id,date)


In [ ]:
###plot example traces

ax2,(ax1,ax2,ax3,ax4) = plt.subplots(4,figsize = (20,10))
onsets = [on/20 for on in events["onsets"]]

sns.lineplot(data = df, x = "Time", y = "Pupil Size", ax = ax1,color = "dodgerblue",linewidth = 1)
sns.lineplot(data = df, x = "Time", y = "Facial Movement", ax = ax2,linewidth = 1,color = "purple")
sns.lineplot(data = df, x = "Time", y = "Running Speed",ax = ax3,linewidth = 1,color = "darkorange")
sns.lineplot(data = df, x = "Time", y = "dFF", ax = ax4,linewidth = 1,color = "green")

for on in onsets:
    ax1.axvline(on,linestyle = "dashed")

x_start = 1510
x_end = 1585
x_space = 5
ylim = [-4,4]

for ax in [ax1,ax2,ax3,ax4]:
    ax.set_xlim(x_start,x_end)
    ax.set_xticks(np.arange(x_start,x_end + x_space,x_space))
    ax.set_ylim(ylim)

ax3.set_ylim(-2,3)
ax1.set_ylim(-2,2)
ax2.set_ylim(-2,5)

ax1.spines[['bottom']].set_visible(False)
ax1.spines[['left']].set_visible(False)

ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)



ax2.spines[['bottom']].set_visible(False)
ax2.spines[['left']].set_visible(False)

ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)



ax3.spines[['bottom']].set_visible(False)
ax3.spines[['left']].set_visible(False)

ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)

ax4.spines[['bottom']].set_visible(False)
ax4.spines[['left']].set_visible(False)

ax4.get_xaxis().set_visible(False)
ax4.get_yaxis().set_visible(False)


ax1.plot([1540,1545], [1, 1],color = "black",linewidth = 1)

ax1.plot([1512,1512], [0, 1],color = "dodgerblue",linewidth = 1)
ax2.plot([1512,1512], [0, 2],color = "purple",linewidth = 1)
ax3.plot([1512,1512], [0, 1],color = "darkorange",linewidth = 1)
ax4.plot([1512,1512], [0, 2],color = "green",linewidth = 1)




#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_example_trace.pdf",format ="pdf",transparent = True)



In [ ]:
###get pearson correlations between behavioral variables

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
df_list = []


for id,dates in id_date.items():
    for date in dates:
      face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
      path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
      df = pd.read_csv(path)
      pupil = np.array(df["Pupil Size"])
      time = np.array(df["Time"])
      speed = (np.array(df["Running Speed"]))
      speed[0] = 0
      speed = zscore(speed)

      dff = np.array(df["dFF"])
      face = np.array(pd.read_csv(face_path)["Facial Movement"])
      df["Facial Movement"] = face

      new_dict = {"R":pearsonr(face,pupil)[0], "type": "F-P", "ID" : id, "Date": date}
      df_list.append(new_dict)
      # new_dict = {"R":pearsonr(face,speed)[0], "type": "F-S", "ID" : id, "Date": date}
      # df_list.append(new_dict)
      new_dict = {"R":pearsonr(speed,pupil)[0], "type": "S-P", "ID" : id, "Date": date}
      df_list.append(new_dict)


r_df = pd.DataFrame.from_dict(df_list)

plt.figure(figsize = (1,1.2))
g = sns.barplot(data = r_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "R",errorbar="se",fill = None,errwidth= 1)
sns.stripplot(data = r_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "R",color = "black",alpha = 0.3,s = 4,jitter = 0.1)

g.set_yticks(np.arange(-0.2,0.8,0.2))
g.set(ylim=(-0.2, 0.6))
g.xaxis.label.set_visible(False)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_corr_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
### do lag analysis based on avg pupil-onset aligned traces
pre = 2
post =2

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
graph_df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        amplitudes = events["amplitudes"]

        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = (np.array(df["Running Speed"]))
        speed = zscore(speed)

        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = pre, post = post)
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = pre, post = post)
        wheel_matrix,wheel_times = trial_align(onsets,time,speed,fps = 20,pre = pre, post = post)

        corr = correlate(np.mean(wheel_matrix,axis=0), np.mean(dil_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(wheel_matrix,axis=0)), len(np.mean(dil_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"S-P"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)" : lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"S-P"}
            graph_df_list.append(graph_dict)


        corr = correlate(np.mean(face_matrix,axis=0), np.mean(dil_matrix,axis = 0))
        lags = correlation_lags(len(np.mean(face_matrix,axis=0)), len(np.mean(dil_matrix,axis = 0)))
        lags = lags/20
        corr /= np.max(corr)
        i = np.argmax(corr)
        delay = lags[i]
        lag_dict = {"Cross-Correlation Delay (s)" : delay,"ID": id, "date" : date,"type":"F-P"}
        df_list.append(lag_dict)

        for lag,c in zip(lags,corr):
            graph_dict = {"Lags (s)" : lag,"Cross-Correlation": c,"ID": id, "date" : date,"type":"F-P"}
            graph_df_list.append(graph_dict)

lag_df = pd.DataFrame.from_dict(df_list)
graph_df = pd.DataFrame.from_dict(graph_df_list)


###generate delay bar graph
plt.figure(figsize = (1,1.2))
g = sns.barplot(data = lag_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Cross-Correlation Delay (s)",errorbar= "se",fill = None,errwidth= 1)
sns.stripplot(data = lag_df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "Cross-Correlation Delay (s)",color = "black",alpha = 0.3,s = 3,jitter = 0.2)


g.set_yticks(np.arange(-1,4,1))
g.set(ylim=(-1, 3))
g.xaxis.label.set_visible(False)
g.set_ylabel("Cross-Correlation\nDelay (s)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_lag_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###cross correlation line graph
plt.figure(figsize = (1,1.2))
ax = sns.lineplot(data = graph_df.groupby(["ID",'type','Lags (s)']).mean().reset_index(), x = "Lags (s)", y = "Cross-Correlation",errorbar="se",hue = "type",palette = ["purple","darkorange"],legend=None,linewidth = 1)
format_ax(ax,(-3,3),(-1,1),3,1)
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_lag_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###Face to pupil median filter
def shuffle(x):
  for i in range(1000):
      x = np.roll(x,randint(0,len(x)-1))
  return np.array(x)
id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
      path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
      df = pd.read_csv(path)
      pupil = np.array(df["Pupil Size"])
      time = np.array(df["Time"])
      speed = zscore((np.array(df["Running Speed"])))
      dff = np.array(df["dFF"])
      face = np.array(pd.read_csv(face_path)["Facial Movement"])
      rs = []
      sizes = list(range(1,120,20))
      for size in sizes:
        smoothed = median_filter(face,size)
        r = pearsonr(smoothed,pupil)[0]
        rs.append(r)
        new_dict = {"R":r, "Window": size/20, "ID" : id, "Date": date,"type" : "F-P (Smoothed Face)"}
        df_list.append(new_dict)

        smoothed = median_filter(pupil,size)
        r = pearsonr(smoothed,face)[0]
        new_dict = {"R":r, "Window": size/20, "ID" : id, "Date": date, "type" : "F-P (Smoothed Pupil)"}
        df_list.append(new_dict)

  
median_df = pd.DataFrame.from_dict(df_list)

In [ ]:
median_df.groupby(["ID","Window","type"]).mean().reset_index()
median_df.groupby(["ID","Window","type"]).mean().reset_index().to_csv("fig1anova.csv")


In [ ]:
model = ols('R ~ C(Window) + C(type) + C(Window):C(type)', data=median_df).fit()
sm.stats.anova_lm(model, typ=2)

In [ ]:
###face-smootehd correalation graph
plt.figure(figsize = (1,1.2))
ax = sns.lineplot(data = median_df.groupby(["ID","Window","type"]).mean().reset_index(), x = "Window", y = "R",hue = "type",errorbar= "se",linewidth = 1,palette = ["Purple","dodgerblue"],legend=None)
format_ax(ax,(0,5),(0,0.8),1,0.2)
ax.set_ylabel("Pupil-Face\nCorrelation (R)")
ax.set_xlabel("Window Size (s)")


#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_face_median_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###example trace smoothed face over pupil

id_date = {
  "004115":["20230815"],
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
      path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
      df = pd.read_csv(path)
      pupil = np.array(df["Pupil Size"])
      time = np.array(df["Time"])
      speed = zscore(abs(np.array(df["Running Speed"])))
      dff = np.array(df["dFF"])
      face = np.array(pd.read_csv(face_path)["Facial Movement"])
      rs = []
      sizes = list(range(1,500,10))
      for size in sizes:
        smoothed = median_filter(face,size)
        r = pearsonr(smoothed,pupil)[0]
        rs.append(r)
        new_dict = {"R":r, "Window Size (s)": size, "ID" : id, "Date": date}
        df_list.append(new_dict)
      
      size = sizes[np.argmax(rs)]
      print(size)
      print(max(rs))





    
      


In [ ]:
plt.figure(figsize = (1.3,1.2))
smoothed = median_filter(face,size)
plt.plot(time,smoothed,linewidth = 1,color  = "purple")
plt.plot(time,pupil,linewidth = 1,color = "dodgerblue")
plt.plot(time,face,alpha = 0.15,linewidth = 1,color = "purple")

plt.xlabel('Time (s)')
plt.ylabel('Z-Score')


format_ax(plt.gca(),(100,130),(-2,8),10,2)
plt.gca().set_xticklabels(range(0,40,10))

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_example_median_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###get event df with binned amplitudes for histograms

id_date = {
"004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}
#labels = list(range(1,5))
labels = ["1","2","3","4"]

df_list = []
for id,dates in id_date.items():
    for date in dates:
        events = get_pupil_events(id,date)
        events["animal_amp_bin"] = pd.qcut(events["amplitudes"], q=4,labels = labels)
        df_list.append(events)
        

event_df = pd.concat(df_list)



In [ ]:
###duration histogram
plt.figure(figsize = (0.75,1.1))
durations = event_df["durations"]
durations = np.clip(durations,a_max = 30,a_min = 0)
g = sns.histplot(data = durations,bins =[0,5,10,15,20,25,30],stat = "percent",fill =False,color = "dodgerblue",linewidth = 1)
g.set(xlabel ="Dilation Duration (s)")
g.set_xticks(np.arange(0,40,5))
g.set(xlim=(0,30))
g.set_yticks(np.arange(0,120,20))
g.set(ylim=(0,100))
labels = [item.get_text() for item in g.get_xticklabels()]
labels[-1] = '30+'
g.set_xticklabels(labels)
g.set_xlabel("Dilation\n Duration (s)")


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dur_hist.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###amplitude histogram
plt.figure(figsize = (0.75,1.1))
amplitudes = event_df["amplitudes"]
amplitudes = np.clip(amplitudes,a_max = 5,a_min = 0)
g = sns.histplot(data = amplitudes,bins =[0,1,2,3,4,5],stat = "percent",fill = False,color = "dodgerblue",linewidth = 1)
g.set(xlabel ="Dilation Amplitude")
g.set_xticks(np.arange(0,6,1))
g.set(xlim=(0,5))
g.set_yticks(np.arange(0,120,20))
g.set(ylim=(0,100))
labels = [item.get_text() for item in plt.gca().get_xticklabels()]
labels[-1] = '5+'
plt.gca().set_xticklabels(labels)
g.set_xlabel("Dilation\nAmplitude (z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_amp_hist.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###amplitude vs duration binned by animal amplitudes
plt.figure(figsize = (0.8,1.1))
# g = sns.pointplot(data = event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(),x = "animal_amp_bin", y = "durations",errwidth = 0.7,scale = 0.3,color = "black")
g = sns.barplot(data = event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(),x = "animal_amp_bin", y = "durations" ,errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.lineplot(data = event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index(), x = "animal_amp_bin", y = "durations",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set(xlabel ="Dilation Amplitude", ylabel = "Dilation Duration")
g.set_yticks(np.arange(0,50,10))
g.set(ylim=(0,40))
g.set_xlabel("Dilation Amplitude\n Quartile")
g.set_ylabel("Dilation\nDuration (s)")


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_dur_amp_scatter.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'")["durations"]
q2 = event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '2'")["durations"]
q3 = event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '3'")["durations"]
q4 = event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '4'")["durations"]
kruskal(q1,q2,q3,q4)

In [ ]:
event_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'")["durations"]

In [ ]:
###speed and face at different pupil quartiles

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

labels = ["1","2","3","4"]
df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        peak_ix = events["peak_ix"]
        onsets = [on for on in events["onsets"]] ###get pupil onsets ix
        offsets = [off for off in events["offsets"]] ###get pupil offsets ix
        amplitudes = events["amplitudes"]
        binned_amps = np.array(pd.qcut(amplitudes,q = 4,labels = labels))

        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        for i,(on,off) in enumerate(zip(onsets,offsets)):
            baseline_face = np.mean(face[on-40:on-10]) 
            baseline_speed = np.mean(speed[on-40:on-10]) 


            during_face = np.mean(face[on:off])
            during_speed = np.mean(speed[on:off])

            new_dict = { "trial" : i,"ID" : id, "Date": date,"amp":amplitudes[i],"animal_amp_bin":binned_amps[i],"∆ Face":during_face-baseline_face,"∆ Speed (z-scr)":during_speed-baseline_speed}
            df_list.append(new_dict)

bar_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###face at different pupil quartiles bar graph
plt.figure(figsize = (1.2,1.2))
g = sns.barplot(data = bar_df.groupby(["ID",'Dilation Amplitude\n Quartile']).mean().reset_index(), x = "Dilation Amplitude\n Quartile", y = "∆ Facial\nMovement (z-scr)",errorbar = "se",fill = None,errwidth= 1,edgecolor = "purple")
sns.lineplot(data = bar_df.groupby(["ID",'Dilation Amplitude\n Quartile']).mean().reset_index(), x = "Dilation Amplitude\n Quartile", y = "∆ Facial\nMovement (z-scr)",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(0,1.25,0.25))
g.set(ylim=(0,1))
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_face_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'")["∆ Face"]
q2 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '2'")["∆ Face"]
q3 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '3'")["∆ Face"]
q4 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '4'")["∆ Face"]
kruskal(q1,q2,q3,q4)

In [ ]:
###speed at different pupil quartiles bar graph
plt.figure(figsize = (1.2,1.2))
g = sns.barplot(data = bar_df.groupby(["ID",'Dilation Amplitude\n Quartile']).mean().reset_index(), x = "Dilation Amplitude\n Quartile", y = "∆ Speed (z-scr)",errorbar = "se",fill = None,errwidth= 1,edgecolor = "darkorange")
sns.lineplot(data = bar_df.groupby(["ID",'Dilation Amplitude\n Quartile']).mean().reset_index(), x = "Dilation Amplitude\n Quartile", y = "∆ Speed (z-scr)",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(-0.2,0.8,0.2))
g.set(ylim=(-0.2,0.6))
plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_ampbin_speed_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
q1 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '1'")["∆ Speed (z-scr)"]
q2 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '2'")["∆ Speed (z-scr)"]
q3 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '3'")["∆ Speed (z-scr)"]
q4 = bar_df.groupby(["ID",'animal_amp_bin']).mean().reset_index().query("animal_amp_bin == '4'")["∆ Speed (z-scr)"]
kruskal(q1,q2,q3,q4)

In [ ]:
###generate aligned graphs binned by amplitude

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        onsets_ix = events["onsets"]
        amplitudes = events["amplitudes"]
        binned_amps = np.array(pd.qcut(amplitudes,q = 4,labels = [1,2,3,4]))

  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])
        
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(face_matrix):
            baseline = np.mean(face[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(face_times):
                new_dict = {"Time from Dilation Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Facial Movement (z-scr)","ID" : id, "Date": date,"animal_amp_bin":binned_amps[j]}
                df_list.append(new_dict)

        speed_matrix,speed_times = trial_align(onsets,time,speed,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(speed_matrix):
            baseline = np.mean(speed[onsets_ix[j] - 40:onsets_ix[j] - 10])
            trial = trial - baseline
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Dilation Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed (z-scr)","ID" : id, "Date": date,"animal_amp_bin":binned_amps[j]}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)

In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.2,1.2))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Dilation Onset (s)","type","animal_amp_bin"]).mean().reset_index().query("type == 'Facial Movement (z-scr)'").query("animal_amp_bin == [1,4]"), x = "Time from Dilation Onset (s)", y = "value",errorbar = "se",hue = "animal_amp_bin",legend=None,linewidth=1,palette= ["violet","purple"])

ax.axvline(0,linestyle = "--", color = "black",alpha = 0.3)
format_ax(ax,(-1,5),(-0.5,1.5),1,0.5)
ax.set_ylabel("Facial\nMovement (z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_face_quartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###plot different quartiles of pupil aligned to pupil onset
plt.figure(figsize = (1.2,1.2))
ax = sns.lineplot(data = align_df.groupby(["ID","Time from Dilation Onset (s)","type","animal_amp_bin"]).mean().reset_index().query("type == 'Speed (z-scr)'").query("animal_amp_bin == [1,4]"), x = "Time from Dilation Onset (s)", y = "value",errorbar = "se",hue = "animal_amp_bin",legend=None,linewidth=1,palette= ["gold","darkorange"])

ax.axvline(0,linestyle = "--", color = "black",alpha = 0.3)
format_ax(ax,(-1,5),(-0.5,1),1,0.5)
ax.set_ylabel("Speed (z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_speed_quartile_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")



In [ ]:
###running vs quiet pupil dilations

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
  "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
        offsets = [off/20 for off in events["offsets"]] ###get pupil offsets
        amplitudes = events["amplitudes"]


        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])

        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)

        for event in list(range(0,len(onsets))):
            on_ix = np.array(events["onsets"])[event]
            off_ix = np.array(events["offsets"])[event]

            baseline_speed = np.mean(speed[on_ix-40:on_ix-10])
            baseline_face = np.mean(face[on_ix-40:on_ix-10]) 



            face_movement = np.mean(face[on_ix:off_ix])
            wheel = np.mean(speed[on_ix:off_ix])

            if event in move_events:
                type = "Running"
            elif event in quiet_events:
                type = "Quiet"
            else:
                type = "neither"
            
            new_dict = {"ID": id, "date" : date,"type":type,"speed":wheel -baseline_speed,"face":face_movement-baseline_face,"amp":amplitudes[event]}
            df_list.append(new_dict)

df = pd.DataFrame.from_dict(df_list)
df = df.query("type != 'neither'")


In [ ]:
df.query("type == 'Quiet'")["ID"].value_counts()

In [ ]:
df.groupby(["ID",'type']).mean().reset_index()

In [ ]:
plt.figure(figsize = (1.2,1.2))
g = sns.barplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "speed",errorbar = "se",fill = None,errwidth= 1,edgecolor  = "darkorange")
sns.lineplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "speed",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(-0.1,0.8,0.1))
g.set(ylim=(-0.1, 0.7))
g.set_ylabel("∆ Speed (z-scr)")
g.xaxis.label.set_visible(False)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_wheel_speed_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = df.groupby(["ID",'type']).mean().reset_index()
group1 = stat_df[stat_df['type']=="Quiet"]
group2 = stat_df[stat_df['type']=="Running"]
wilcoxon(group1["speed"], group2["speed"],nan_policy= "omit",method = 'approx')

In [ ]:
plt.figure(figsize = (1.2,1.2))
g = sns.barplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "face",errorbar = "se",fill = None,errwidth= 1,edgecolor = "purple")
sns.lineplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "face",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(-1,1.5,0.5))
g.set(ylim=(0, 1))
g.set_ylabel("∆ Facial\nMovement (z-scr)")
g.xaxis.label.set_visible(False)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_wheel_face_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = df.groupby(["ID",'type']).mean().reset_index()
group1 = stat_df[stat_df['type']=="Quiet"]
group2 = stat_df[stat_df['type']=="Running"]
wilcoxon(group1["face"], group2["face"],nan_policy= "omit")

In [ ]:
plt.figure(figsize = (1.2,1.2))
g = sns.barplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "amp",errorbar = "se",fill = None,errwidth= 1,edgecolor = "dodgerblue")
sns.lineplot(data = df.groupby(["ID",'type']).mean().reset_index(), x = "type", y = "amp",units = "ID",estimator = None,color = "black",alpha = 0.3,linewidth = 0.5)

g.set_yticks(np.arange(0,4,1))
g.set(ylim=(0, 3))
g.set_ylabel("Dilation\nAmplitude (z-scr)")
g.xaxis.label.set_visible(False)


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_wheel_amp_bar.pdf",format ="pdf",transparent = True,bbox_inches = "tight")


In [ ]:
###stats
stat_df = df.groupby(["ID",'type']).mean().reset_index()
group1 = stat_df[stat_df['type']=="Quiet"]
group2 = stat_df[stat_df['type']=="Running"]
wilcoxon(group1["amp"], group2["amp"],nan_policy= "omit")

In [ ]:
df.groupby(["ID",'type']).mean().reset_index()

In [ ]:
###generate movement vs non movement aligned traces

id_date = {
  "004113":["20230808"],
  "004114":["20230808","20230815"],
   "004115":["20230804","20230808","20230815"],
  "004116":["20230804","20230808","20230815"],
  "004117":["20230804","20230808","20230815"],
  "004118":["20230804","20230808","20230815"]
}

df_list = []
for id,dates in id_date.items():
    for date in dates:
      
        events = get_pupil_events(id,date)
        onsets = [on/20 for on in events["onsets"]] ###get pupil onsets
  
        face_path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_FaceProcessed.csv"
        path = f"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/ethanol_data/{id}/{date}/{date}_{id}_sess_1_ProcessedData.csv"
        df = pd.read_csv(path)
        pupil = np.array(df["Pupil Size"])
        time = np.array(df["Time"])
        speed = zscore((np.array(df["Running Speed"])))
        raw_speed = np.array(df["Running Speed"])
     
        dff = np.array(df["dFF"])
        face = np.array(pd.read_csv(face_path)["Facial Movement"])

        move_events = get_movement_events(events,raw_speed)
        quiet_events = get_quiet_events(events,raw_speed)

        dil_matrix,dil_times = trial_align(onsets,time,pupil,fps = 20,pre = 10, post = 10 )

        for j,trial in enumerate(dil_matrix):
            baseline = np.mean(trial[160:190])
            trial = trial-baseline
            if j in quiet_events:
                type = "quiet"
            elif j in move_events:
                type = "running"
            else:
                type = "neither"
            for i,Time in enumerate(dil_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Pupil","ID" : id, "Date": date,"Speed":type}
                df_list.append(new_dict)
                
        face_matrix,face_times = trial_align(onsets,time,face,fps = 20,pre = 10, post = 10)
        for j,trial in enumerate(face_matrix):
            baseline = np.mean(trial[160:190])
            trial = trial-baseline
            if j in quiet_events:
                type = "quiet"
            elif j in move_events:
                type = "running"
            else:
                type = "neither"
            for i,Time in enumerate(face_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Face","ID" : id, "Date": date,"Speed":type}
                df_list.append(new_dict)

        speed_matrix,speed_times = trial_align(onsets,time,speed,fps = 20,pre = 10, post = 10)

        for j,trial in enumerate(speed_matrix):
            baseline = np.mean(trial[160:190])
            trial = trial-baseline
            if j in move_events:
                type = "running"
            elif j in quiet_events:
                type = "quiet"
            else:
                type = "neither"
            for i,Time in enumerate(speed_times):
                new_dict = {"Time from Onset (s)" : Time, "trial" : j, "value" : trial[i], "type" : "Speed","ID" : id, "Date": date,"Speed":type}
                df_list.append(new_dict)

align_df = pd.DataFrame.from_dict(df_list)
align_df = align_df.query("Speed != 'neither'")


In [ ]:
plt.figure(figsize = (1.2,1.2))

ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","Speed"]).mean().reset_index().query("type == 'Pupil'"), x = "Time from Onset (s)", y = "value", hue = "Speed",errorbar = "se",legend = None,linewidth = 1, palette= ["cornflowerblue","darkblue"])

ax.axvline(0,linestyle = "dashed", color = "black",alpha = 0.3)
format_ax(ax,(-1,5),(-0.5,1.5),1,0.5)
ax.set_xlabel("Time from Dilation Onset (s)")
ax.set_ylabel("Pupil Size (z-scr)")

plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_speedbin_pupil_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
plt.figure(figsize = (1.2,1.2))

ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","Speed"]).mean().reset_index().query("type == 'Face'"), x = "Time from Onset (s)", y = "value", hue = "Speed",errorbar = "se",legend = None,linewidth = 1, palette= ["violet","purple"])

ax.axvline(0,linestyle = "dashed", color = "black",alpha = 0.3)
format_ax(ax,(-1,5),(-0.5,1.5),1,0.5)

ax.set_xlabel("Time from Dilation Onset (s)")
ax.set_ylabel("Facial\nMovement (z-scr)")


plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_speedbin_face_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")

In [ ]:
plt.figure(figsize = (1.2,1.2))

ax = sns.lineplot(data = align_df.groupby(["ID","Time from Onset (s)","type","Speed"]).mean().reset_index().query("type == 'Speed'"), x = "Time from Onset (s)", y = "value", hue = "Speed",errorbar = "se",legend = None,linewidth = 1, palette= ["gold","darkorange"])

ax.axvline(0,linestyle = "dashed", color = "black",alpha = 0.3)
format_ax(ax,(-1,5),(-0.5,1),1,0.5)

ax.set_xlabel("Time from Dilation Onset (s)")
ax.set_ylabel("Speed (z-scr)")


#plt.savefig(rf"/Users/nithik/Library/CloudStorage/Box-Box/HUDA_LAB_DATA/Manuscripts/2023-ACCArousal-CurrentBiology/SavedGraphs/FP_speedbin_speed_line.pdf",format ="pdf",transparent = True,bbox_inches = "tight")